In [10]:
# to print all output for a cell instead of only last one 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [11]:
# IMPORTS 

from pathlib import Path 
import os 

import pandas as pd
import numpy as np
from collections import OrderedDict, namedtuple

In [12]:
# PATHS 
DATA_FOLDER = Path(*Path().absolute().parts[:-1]) / 'data' # directory containing the notebook

Load data


In [4]:
import json 

json_file_path_train = DATA_FOLDER / 'Twibot-20/train.json'
json_file_path_val = DATA_FOLDER / 'Twibot-20/dev.json'
json_file_path_test = DATA_FOLDER / 'Twibot-20/test.json'

with open(json_file_path_train, 'r') as tr:
     contents = json.loads(tr.read())
     train_df = pd.json_normalize(contents)
     train_df['split'] = 'train'

with open(json_file_path_val, 'r') as vl:
     contents = json.loads(vl.read())
     val_df = pd.json_normalize(contents) 
     val_df['split'] = 'val'

with open(json_file_path_test, 'r') as ts:
     contents = json.loads(ts.read())
     test_df = pd.json_normalize(contents) 
     test_df['split'] = 'test'

df = pd.concat([train_df,val_df,test_df],ignore_index=True) # merge three datasets
df.dropna(subset=['tweet'], inplace=True)  # remove rows withot any tweet 
df.set_index(keys='ID',inplace=True) # reset index

# split dataframe in two : tweet and account data 
tweets_df = df[['tweet','label','split']].reset_index()
tweets_df = tweets_df.explode('tweet').reset_index(drop=True)
tweets_df.rename(columns={"ID": "account_id"}, inplace=True)

account_df = df.drop('tweet',axis=1).reset_index()
account_df.rename(columns={"ID": "account_id"}, inplace=True)


DATA PROCESSING AND CLEANING  

In [13]:
import emoji
from nltk.tokenize import TweetTokenizer

from ttp import ttp 
parser = ttp.Parser(include_spans=True)

from emot.core import emot
emot_obj = emot()

import re 

tk = TweetTokenizer(reduce_len=True,preserve_case=False)

CASHTAG = "(?<!\S)\$[A-Z]+(?:\.[A-Z]+)?(?!\S)"   # to check  (?:\.[A-Z]+)?
EMAIL = r"""[\w.+-]+@[\w-]+\.(?:[\w-]\.?)+[\w-]"""
MONEY = "[$£][0-9]+(?:[.,]\d+)?[k+B]?|[0-9]+(?:[.,]\d+)?[k+B]?[$£]"  
NUMBERS = r"""(?<!\S)(?:[+\-]?\d+(?:%|(?:[,/.:-]\d+[+\-]?)?))"""   # r"""(?:[+\-]?\d+[,/.:-]\d+[+\-]?)"""   
HASHTAG = r"""(?:\#+[\w_]+[\w\'_\-]*[\w_]+)"""
HANDLES = r"""(?:@[\w_]+)""" 

TO_REPLACE = [CASHTAG, EMAIL, MONEY, NUMBERS, HASHTAG, HANDLES]
REPLACE_WITH = [' stock ',' email ',' money ',' number ',' hashtag ',' username ']


def replace(word : str):
    if not word.isascii():
        return ['']
    elif bool(re.search(r'http[s]?|.com',word)):
        return ['url']
    elif bool(re.search(r'\d',word)):
        return ['number']
    elif bool(re.search(r'haha|ahah|jaja|ajaj',word)):
        return ['ahah']
    elif bool(re.search(r'\n',word)):
        return ['']
    elif bool(re.search('-',word)):
        return re.sub('-',' ',word).split()    #TODO separare parole con l'apostrofo
    else :
        return [word] 
    

def further_process(sentence: str):
        #replace urls 
        result = parser.parse(sentence, html=False)
        urls = dict(result.urls).keys()
        for url in urls:
                sentence = sentence.replace(url,' url ')
        
        #replace emoticons 
        emoticons = emot_obj.emoticons(sentence)
        for emoticon in emoticons['value']:
                sentence = sentence.replace(emoticon,' emoticon ')
        
        #replace emoji
        sentence = emoji.replace_emoji(sentence,' emoji ')

        #tokenize
        sentence = tk.tokenize(sentence)

        #replace residual wrong words 
        sentence = [w for word in sentence for w in replace(word)]
        
        #remove empty strings 
        sentence = [word for word in sentence if word != '']
                
        return sentence

dataset_path = DATA_FOLDER / 'processed_dataset.pkl'
force_processing = False

#apply preprocessing      
if os.path.exists(dataset_path) and not force_processing: 
    print('found already processed dataset in data folder, retrieving the file...')
    tweets_df = pd.read_pickle(dataset_path)
    print('dataset loaded in Dataframe')
    
else : 
    tweets_df['processed_tweet'] = tweets_df['tweet'].replace(TO_REPLACE,REPLACE_WITH,regex=True,inplace=False)
    tweets_df['processed_tweet'] = tweets_df['processed_tweet'].apply(further_process)

    tweets_df[tweets_df['processed_tweet'].map(lambda x: len(x)) > 0].reset_index(drop=True,inplace=True)   #TODO CHECK  

    tweets_df['label'] = tweets_df['label'].astype(float)  #TODO CHECK   # transform label from string to float 

    tweets_df.to_pickle(DATA_FOLDER / 'processed_dataset.pkl')   #save to file 


found already processed dataset in data folder, retrieving the file...
dataset loaded in Dataframe


DOWNLOAD TWITTER GLOVE EMBEDDINGS

In [22]:
import gensim
import gensim.downloader as gloader
from gensim.models import KeyedVectors

glove_model_cached_path = DATA_FOLDER / 'glove_vectors.txt'
glove_model_download_path = 'glove-twitter-200'
force_download = False  # to download glove model even if the vectors model has been already stored. Mainly for testing purposes

if os.path.exists(glove_model_cached_path) and not force_download: 
    print('found cached glove vectors in data folder, retrieving the file...')
    emb_model = KeyedVectors.load_word2vec_format(glove_model_cached_path, binary=True)
    print('vectors loaded')

else:
    print('downloading glove embeddings...')        
    emb_model = gloader.load(glove_model_download_path)

    print('saving glove embeddings to file')  
    emb_model.save_word2vec_format(glove_model_cached_path, binary=True)

found cached glove vectors in data folder, retrieving the file...
vectors loaded


CUSTOM DATA HANDLING  

In [62]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.utils.rnn as rnn
import torch 

Vocab = namedtuple('Vocabulary',['word2int','int2word','unique_words'])

class TwitterDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame):
        self.tweet = dataframe['processed_tweet']
        self.label = dataframe['label']

    def __len__(self):
        return len(self.label)

    def __getitem__(self, idx):
        return {
            'tweet': self.tweet[idx],
            'label': self.label[idx],
            }

class TwitterDataManager():

    def __init__(self, dataframe : pd.DataFrame, device ):

        self.device = device 

        self.dataset = dataframe.copy(deep=True)
        self.train_ds = TwitterDataset(self.dataset[self.dataset['split'] == 'train'].reset_index(drop=True))
        self.val_ds = TwitterDataset(self.dataset[self.dataset['split'] == 'val'].reset_index(drop=True))
        self.test_ds = TwitterDataset(self.dataset[self.dataset['split'] == 'test'].reset_index(drop=True))

    def custom_collate(self, batch):
        
        tweet_lengths = torch.tensor([len(example['tweet']) for example in batch]) #, device=self.device -> for pack_padded should be on cpu so if only used by that don't put it on gpu

        numerized_tweets = [self.numericalize(example['tweet']) for example in batch]
        padded_tweets = rnn.pad_sequence(numerized_tweets, batch_first = True, padding_value = self.vocab.word2int['<pad>']).to(self.device)

        labels = torch.tensor([example['label'] for example in batch],device=self.device) #(5)

        return {
            'tweets': padded_tweets,
            'labels': labels,
            'lengths': tweet_lengths
        }
    
    def numericalize(self, token_list):  

        assert self.vocab is not None, "you have to build the vocab first, call build_vocab method to do it"
        return torch.tensor(list(map(self.vocab.word2int.get,token_list)))
    
    def build_vocab(self): 
        print('Building vocab...')

        unique_words : list = self.dataset['processed_tweet'].explode().unique().tolist()
        unique_words.insert(0,'<pad>')

        word2int = OrderedDict()
        int2word = OrderedDict()

        for i, word in enumerate(unique_words):
            word2int[word] = i           
            int2word[i] = word
        
        self.vocab = Vocab(word2int,int2word,unique_words)

        print(f'the number of unique words is {len(unique_words)}')
    
    def build_emb_matrix(self, emb_model): 
        print('Building embedding matrix...')

        embedding_dimension = emb_model.vector_size #how many numbers each emb vector is composed of                                                           
        embedding_matrix = np.zeros((len(self.vocab.word2int)+1, embedding_dimension), dtype=np.float32)   #create a matrix initialized with all zeros 

        for word, idx in self.vocab.word2int.items():
            try:
                embedding_vector = emb_model[word]
            except (KeyError, TypeError):
                embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

            embedding_matrix[idx] = embedding_vector     #assign the retrived or the generated vector to the corresponding index 
        
        self.emb_matrix = embedding_matrix
        
        print(f"Embedding matrix shape: {embedding_matrix.shape}")
    
    def getDataloader(self, split : str, batch_size : int, shuffle : bool):

        dataset = getattr(self,split+'_ds') 
        return DataLoader(dataset,batch_size,shuffle=shuffle,collate_fn=self.custom_collate)

MODEL : SINGLE TWEET NETWORK 

In [85]:
#pytorch imports

import torch.nn as nn
from torch import Tensor
import torch.optim as optim
import torch.nn.utils.rnn as rnn
import torch.nn.functional as F

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence



In [88]:
class SingleTweet_model(nn.Module):
    
    def __init__(self, emb_matrix: np.ndarray, cfg : dict) :
        super().__init__()

        self.cfg = cfg

        self.embedding_layer, self.word_embedding_dim = self.build_emb_layer(emb_matrix,cfg['pad_idx'], cfg['freeze_embedding'])

        self.lstm = nn.LSTM(self.word_embedding_dim, cfg['hidden_dim'], batch_first = True, bidirectional = True) 
            
        self.dropout = nn.Dropout(cfg['dropout_p']) 

        self.compress = nn.Linear(cfg['hidden_dim']*2,cfg['hidden_dim'])

        self.classifier = nn.Linear(cfg['hidden_dim'],1)   
    
    def name():
        return 'SingleTweet_model'


    def build_emb_layer(self, weights_matrix: np.ndarray, pad_idx : int, freeze = True):
    
        matrix = torch.Tensor(weights_matrix, device = self.device)   #the embedding matrix 
        _ , embedding_dim = matrix.shape 

        emb_layer = nn.Embedding.from_pretrained(matrix, freeze=freeze, padding_idx = pad_idx)   #load pretrained weights in the layer and make it non-trainable (TODO: trainable ? )
        
        return emb_layer, embedding_dim
        

    def forward(self, batch_data):
    
        tweets = batch_data['tweets']           # [batch_size, num_tokens]
        tweet_lengths = batch_data['lengths']   # [batch_size]

        #embed each word in a sentence with a 200d vector 
        word_emb_tweets = self.embedding_layer(tweets)  # word_emb_tweets = [batch_size, num_tokens, embedding_dim]

        #pass the embedded tokens throught lstm network 
        packed_embeddings = pack_padded_sequence(word_emb_tweets, tweet_lengths, batch_first=True, enforce_sorted=False) #tweet_lengths.cpu() TODO
        output, (hn,cn)  = self.lstm(packed_embeddings)   # hn = [2, batch_size, embedding_dim]
        
        #concat forward and backward output
        fwbw_hn = torch.cat(hn[-1,:,:],hn[-2,:,:],dim=1)  # fwbw_hn = [batch_size, 2*embedding_dim]
        
        #compress the output 
        compressed_out = self.compress(fwbw_hn) # compressed_out = [batch_size, embedding_dim]

        #apply non linearity
        compressed_out = F.relu(compressed_out)

        #eventual dropout 
        if self.cfg['dropout']: sentence_emb_tweet = self.dropout(compressed_out)

        #final classification 
        predictions = self.classifier(sentence_emb_tweet) #predictions [batch_size, 1]

        return predictions

In [ ]:
import time 
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm

#compute accuracy and f1-score 
def acc_and_f1(y_true, y_pred):
    """
        Compute accuracy and f1-score for an epoch 
    """
    acc = accuracy_score(y_true, y_pred)

    f1 = f1_score(y_true,y_pred,average='macro')

    return acc, f1

class Trainer():

    def __init__(self, model : nn.Module, device, criterion, optimizer) : #TODO qualcosa 
        
        self.device = device 

        model.to(self.device)
        self.model = model

        self.criterion = criterion.to(self.device) if isinstance(criterion, nn.Module) else criterion 
        self.optimizer = optimizer


    def train_loop(self, dataloader : DataLoader):

        start = time.perf_counter()

        tot_loss = 0
        
        #aggregate all the predictions and corresponding true labels (and claim ids) in tensors 
        all_pred , all_targ = np.empty(len(dataloader)), np.empty(len(dataloader))

        self.model.train()
    
        for batch_id, batch_data in enumerate(tqdm(dataloader)):

            self.optimizer.zero_grad()            

            predictions : Tensor = self.model(batch_data)   #generate predictions 

            loss = self.criterion(predictions.squeeze(0), batch_data['labels'])      #compute the loss 

            #backward pass 
            loss.backward()
            self.optimizer.step()

            pred = (predictions > 0.0 ).int().cpu().numpy()           #get class label 

            start = batch_id * dataloader.batch_size
            end = start + dataloader.batch_size

            #concatenate the new tensors with the one computed in previous steps
            all_pred[start:end] = pred 
            all_targ[start:end] = batch_data['labels'].numpy()

            tot_loss += loss.item()    #accumulate batch loss 


        acc, f1 = acc_and_f1(all_targ,all_pred)

        loss = tot_loss/(batch_id+1)    #mean loss 

        end = time.perf_counter()

        return loss, acc, f1


    def eval_loop(self, dataloader):
        
        start = time.perf_counter()

        tot_loss = 0
        
        #aggregate all the predictions and corresponding true labels (and claim ids) in tensors 
        all_pred , all_targ = np.empty(len(dataloader)), np.empty(len(dataloader))
        
        self.model.eval()   #model in eval mode 
        
        with torch.no_grad(): #without computing gradients since it is evaluation loop
        
            for batch_id, batch_data in enumerate(tqdm(dataloader)):
                
                predictions : Tensor = self.model(batch_data)   #generate predictions 

                loss = self.criterion(predictions.squeeze(0), batch_data['labels'])      #compute the loss 

                pred = (predictions > 0.0 ).int().cpu().numpy()        #get class label 
                start = batch_id * dataloader.batch_size
                end = start + dataloader.batch_size

                #concatenate the new tensors with the one computed in previous steps
                all_pred[start:end] = pred 
                all_targ[start:end] = batch_data['labels'].numpy()

                tot_loss += loss.item()   #accumulate batch loss 
                
        acc, f1 = acc_and_f1(all_targ,all_pred)

        loss = tot_loss/(batch_id+1)   #mean loss 

        end = time.perf_counter()

        return loss, acc, f1

    
    def train_and_eval(self, train_loader, val_loader, num_epochs):
        """
            Runs the train and eval loop and keeps track of all the metrics of the training model 
        """
        best_f1 = -1   #init best f1 score

        for epoch in range(1, num_epochs): #epoch loop

            start_time = time.perf_counter()

            print(f'Starting epoch {epoch}')

            train_metrics = self.train_loop(train_loader) 
            val_metrics = self.eval_loop(val_loader) 
            
            end_time = time.perf_counter()

            tot_epoch_time = end_time-start_time          

            train_epoch_loss, train_epoch_acc, train_epoch_f1 = train_metrics
            val_epoch_loss, val_epoch_acc, val_epoch_f1 = val_metrics

            if val_epoch_f1 >= best_f1:
                best_f1 = val_epoch_f1
                if not os.path.exists('models'):        
                    os.makedirs('models')
                torch.save(self.model.state_dict(),f'models/{self.model.name()}.pt')  

            #TODO add wandb log 
        
            print(f'Total epoch Time: {tot_epoch_time:.4f}')
            print(f'\tTrain Loss: {train_epoch_loss:.3f} | Train Acc: {train_epoch_acc*100:.2f}% | Train F1: {train_epoch_f1:.2f}')
            print(f'\t Val. Loss: {val_epoch_loss:.3f} | Val. Acc: {val_epoch_acc*100:.2f}% | Val. F1: {val_epoch_f1:.2f}')
        

Single Tweet Model Train Routine 

In [ ]:
#PARAMETERS, HYPERPARAMETERS AND USEFUL OBJECTS 

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'running on {DEVICE}')

#hyperparameters
BATCH_SIZE = 512                # number of sentences in each mini-batch
LR = 1e-3                       # learning rate 
NUM_EPOCHS = 10                   # number of epochs
WEIGHT_DECAY = 1e-5             # regularization
LSTM_HIDDEN_DIM = 200           # hidden dimension of lstm network 
FREEZE = False                  # wheter to make the embedding layer trainable or not              
DROPOUT = True                  # wheter to use dropout layer or not  
DROPOUT_P = 0.5                 # dropout probability


#to counteract class imbalance 
# (human, bot) = train['label'].value_counts()    #number of supports and refutes in the train dataset 
# weight_positive_class = torch.Tensor([bot/human]).to(DEVICE)  #weight to give to positive class 

data_manager = TwitterDataManager(tweets_df,DEVICE)
data_manager.build_vocab()
data_manager.build_emb_matrix(emb_model)

# model config parameters dictionary
model_cfg = {
    'pad_idx' : data_manager.vocab.word2int('<pad>'),
    'freeze_embedding' : FREEZE,  
    'dropout' : DROPOUT,
    'dropout_p' : DROPOUT_P,
    'hidden_dim' : LSTM_HIDDEN_DIM
}

model = SingleTweet_model(data_manager.emb_matrix,model_cfg)

# criterion = nn.BCEWithLogitsLoss(pos_weight=param['weight_positive_class']).to(device)    #Binary CrossEntropy Loss that accept raw input and apply internally the sigmoid 
criterion = nn.BCEWithLogitsLoss()    #Binary CrossEntropy Loss that accept raw input and apply internally the sigmoid 
optimizer = optim.Adam(model.parameters(), lr=LR ,  weight_decay= WEIGHT_DECAY)   #L2 regularization 

train_loader = data_manager.getDataloader('train', BATCH_SIZE, True)
val_loader = data_manager.getDataloader('val', BATCH_SIZE, True)

trainer = Trainer(model, DEVICE, criterion, optimizer)
trainer.train_and_eval(train_loader, val_loader, NUM_EPOCHS)

In [63]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [64]:
tweets_df['label'] = tweets_df['label'].astype(float)

In [65]:
data_manager = TwitterDataManager(tweets_df,DEVICE)
data_manager.build_vocab()
data_manager.vocab.word2int['<pad>']

Building vocab...
the number of unique words is 397347


0

In [66]:
l = data_manager.getDataloader('train',5,False)

In [ ]:


for batch_id, batch_data in enumerate(l):
    start = batch_id * l.batch_size
    end = start + l.batch_size

    print(start,end)




In [83]:
a = np.empty((20))

a[0:5] = 1.0

a

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])